#Q4
Build a classifier for Street View House Numbers (SVHN) (Dataset) using pretrained model weights from PyTorch. Try multiple
models like LeNet-5, AlexNet, VGG, or ResNet(18, 50, 101). Compare performance comment why a particular model is well suited
for SVHN dataset. (You can use a subset of dataset (25%) in case you do not have enough compute.)

In [1]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torchvision.models as models
import numpy as np
import os



In [2]:
# Random seed for reproducibility
np.random.seed(0)
torch.manual_seed(0)

# Loadind and preprocessing SVHN dataset
transform = transforms.Compose([
    transforms.Resize((32, 32)),  # Resize images to match the input size expected by the models
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # Normalize pixel values to [-1, 1]
])

train_dataset = datasets.SVHN(root='./data', split='train', transform=transform, download=True)
test_dataset = datasets.SVHN(root='./data', split='test', transform=transform, download=True)


train_subset = torch.utils.data.Subset(train_dataset, np.random.choice(len(train_dataset), int(0.25 * len(train_dataset)), replace=False))


100%|██████████| 182040794/182040794 [00:21<00:00, 8301012.09it/s]


100%|██████████| 64275384/64275384 [00:15<00:00, 4225538.68it/s]


In [3]:


download_path = './pretrained_weights/'

# Creating directory if it doesn't exist
os.makedirs(download_path, exist_ok=True)

# Defining VGG16 with pretrained weights
vgg = models.vgg16(pretrained=True)
# Saving the pretrained weights
torch.save(vgg.state_dict(), os.path.join(download_path, 'vgg_pretrained_weights.pth'))

# Defining ResNet-18 with pretrained weights
resnet18 = models.resnet18(pretrained=True)
# Saving the pretrained weights
torch.save(resnet18.state_dict(), os.path.join(download_path, 'resnet18_pretrained_weights.pth'))

# Defining ResNet-50 with pretrained weights
resnet50 = models.resnet50(pretrained=True)
# Saving the pretrained weights
torch.save(resnet50.state_dict(), os.path.join(download_path, 'resnet50_pretrained_weights.pth'))

# Defining ResNet-101 with pretrained weights
resnet101 = models.resnet101(pretrained=True)
# Saving the pretrained weights
torch.save(resnet101.state_dict(), os.path.join(download_path, 'resnet101_pretrained_weights.pth'))


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth
100%|██████████| 528M/528M [00:03<00:00, 151MB/s]
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. 

In [4]:
# Loading pretrained weights

vgg_weights = torch.load('./pretrained_weights/vgg_pretrained_weights.pth')
resnet18_weights = torch.load('./pretrained_weights/resnet18_pretrained_weights.pth')
resnet50_weights = torch.load('./pretrained_weights/resnet50_pretrained_weights.pth')
resnet101_weights = torch.load('./pretrained_weights/resnet101_pretrained_weights.pth')


In [5]:
# Defining loss function and optimizer
criterion = nn.CrossEntropyLoss()


In [7]:
# Defining data loaders
train_loader = torch.utils.data.DataLoader(train_subset, batch_size=64, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=64, shuffle=False)


In [13]:
def train_and_evaluate(model, train_loader, test_loader, criterion, optimizer, num_epochs=5, device=torch.device("cuda")):
    model.to(device)
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss / len(train_loader)}")

    model.eval()
    total_correct = 0
    total_samples = 0
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            total_correct += (predicted.cpu() == labels.cpu()).sum().item()  # Move predicted and labels back to CPU for accuracy calculation
            total_samples += labels.size(0)
    accuracy = total_correct / total_samples
    print(f"Accuracy: {accuracy}\n")

# Training and evaluating models
models = [vgg, resnet18, resnet50, resnet101]
model_names = ['VGG-16', 'ResNet-18', 'ResNet-50', 'ResNet-101']

for model, model_name in zip(models, model_names):
    print(f'Training and evaluating {model_name}...\n')
    optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
    train_and_evaluate(model, train_loader, test_loader, criterion, optimizer, device=torch.device("cuda"))


Training and evaluating VGG-16...

Epoch 1/5, Loss: 2.288294100595268
Epoch 2/5, Loss: 1.058486030284536
Epoch 3/5, Loss: 0.49857990267177077
Epoch 4/5, Loss: 0.35887506611982706
Epoch 5/5, Loss: 0.2855098379795352
Accuracy: 0.9094960049170252

Training and evaluating ResNet-18...

Epoch 1/5, Loss: 2.0546019673347473
Epoch 2/5, Loss: 0.7374378602679182
Epoch 3/5, Loss: 0.5432222432064262
Epoch 4/5, Loss: 0.3694945172981103
Epoch 5/5, Loss: 0.29688645213946235
Accuracy: 0.8845267363245236

Training and evaluating ResNet-50...

Epoch 1/5, Loss: 2.3068630649653046
Epoch 2/5, Loss: 1.5649826250425198
Epoch 3/5, Loss: 1.01850483984482
Epoch 4/5, Loss: 0.8195795713816786
Epoch 5/5, Loss: 0.7497649172041889
Accuracy: 0.7956745543945912

Training and evaluating ResNet-101...

Epoch 1/5, Loss: 2.6094645938806833
Epoch 2/5, Loss: 2.429790206902534
Epoch 3/5, Loss: 2.3804311278805086
Epoch 4/5, Loss: 2.3331697888490632
Epoch 5/5, Loss: 2.094680352078082
Accuracy: 0.4108020897357099

